<a href="https://colab.research.google.com/github/HJbgk12/1.-AI-Goorm-Text-Classification/blob/master/Text_Classification(alone).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade protobuf

     |████████████████████████████████| 1.1 MB 3.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3


In [2]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 180 kB 64.5 MB/s 
     |████████████████████████████████| 143 kB 73.7 MB/s 
     |████████████████████████████████| 97 kB 9.7 MB/s 
     |████████████████████████████████| 63 kB 2.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=38418321d5c85f5fa27c90d89a6e8f228ea0f304f69ca658c39216107366862a
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=929dd02fd20df7b7981512e8e0cdd3f866e843ee3aab5c2d016af444fe7dbce9
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [3]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 895 kB 63.7 MB/s 
     |████████████████████████████████| 596 kB 67.5 MB/s 
     |████████████████████████████████| 6.8 MB 57.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import os
import pdb
# import wandb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange


from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

# **Pre Processing**#

In [5]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [6]:
nltk.download('stopwords')
print('영어 불용어 갯수:',len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:40])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
영어 불용어 갯수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this']


In [140]:
print(nltk.corpus.stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
def del_stopword(data):
    stop_words = set(stopwords.words('english'))
    result = []
    word_list = data.split(' ')
    for token in word_list:
        if token not in stop_words:
            result.append(token)
            
    return result

In [8]:
def data_text_cleaning(data):
    spacing = data.replace("'s", ' is')
    spacing = spacing.replace("n't", " not")
    spacing = spacing.replace("'re", ' are')
    spacing = spacing.replace("'ll", ' will')

    # 영문자 이외 문자는 공백으로 변환
    only_english = re.sub('[^a-zA-Z]', ' ', spacing)

    # 소문자 변환
    no_capitals = only_english.lower().split()

    # print(' '.join(no_capitals))
    no_stopword = del_stopword(' '.join(no_capitals))
    
    return no_stopword

#**Tokenize**#

In [9]:
def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join('sample_data/', file_name), 'r', encoding='utf-8') as f:
            pre_data = [data_text_cleaning(line.strip()) for line in f.readlines()]
            pre_data = [line for line in pre_data if line]
            id_file_data = [tokenizer.encode(line) for line in pre_data]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    train_pos = make_data_strings('sentiment.train.1')
    train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [13]:
train_pos[:10]

['101 6581 2833 102',
 '101 21688 8013 2326 102',
 '101 2036 3679 19247 3256 6949 2428 2204 102',
 '101 2204 100 100 102',
 '101 3095 5379 102',
 '101 2204 3347 2833 102',
 '101 2204 2326 102',
 '101 11350 2154 25628 7167 19247 102',
 '101 2307 2173 6265 3347 27962 5404 102',
 '101 2047 2846 3504 6429 102']

train_pos 데이터와 train_neg 데이터의 라벨링을 해준다.

train_pos => 1 / train_neg => 0

In [14]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [15]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

각각의 dataset은 token화된 arraylist와 라벨링된 arraylist로 이루어져있다.

In [16]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

(array([ 101, 6581, 2833,  102]), array([1]))
(array([  101, 21688,  8013,  2326,   102]), array([1]))
(array([  101,  2036,  3679, 19247,  3256,  6949,  2428,  2204,   102]), array([1]))
(array([ 101, 2204,  100,  100,  102]), array([1]))
(array([ 101, 3095, 5379,  102]), array([1]))
(array([ 101, 2204, 3347, 2833,  102]), array([1]))
(array([ 101, 2204, 2326,  102]), array([1]))
(array([  101, 11350,  2154, 25628,  7167, 19247,   102]), array([1]))
(array([  101,  2307,  2173,  6265,  3347, 27962,  5404,   102]), array([1]))
(array([ 101, 2047, 2846, 3504, 6429,  102]), array([1]))
(array([ 101, 2173, 2204,  102]), array([1]))


In [17]:
def collate_fn_sentiment(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1] # 데이터 길이순으로 정렬

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True) # 배치사이즈마다의 max_sequance_len을 기준으로 패딩처리를 해줌 : 빨라짐
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

배치 사이즈를 정의해야 하지만 성능의 상관관계는 데이터에 따라 기준이 달라지기 때문에 아직 정확히 규정된 바 없다.

다만 일반적으로 32,64의 배치사이즈가 성능에 가장 좋다고 알려져있다.

배치사이즈를 줄일 시 : 최소 요구 메모리량이 감소한다. 

배치사이즈를 늘릴 시 : 적은 배치사이즈로 학습할 때 보다 덜 flucatuate(불안정)해진다.

num_workers는 학습 도중 CPU의 작업을 몇개의 코어를 사용해서 진행할 지에 대한 파라미터이다.

In [18]:
!nvidia-smi

Sun Jan 30 03:01:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [116]:
train_batch_size=128
eval_batch_size=128

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_sentiment,
                                           pin_memory=True, num_workers=4)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_sentiment,
                                         num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#**Model**#

Bert 모델을 생성하여 분류한다.

In [117]:
# random seed
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

# wandb로 추적
# wandb.config = {"learning_rate" : 5e-5, "epochs" : 1, "batch_size" : 64}
# wandb.watch(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

왜 AdamW를 사용하였는가?

=> L2 regularization term이 추가된 loss func을 Adam을 이용해서 최적화 할 경우, 의도와 달리 일반화 효과가 떨어질 수 있다고 한다.
Adam은 adaptive learning rate을 위해

그래디언트 제곱 누적 값의 역수에 루트를 취한 것으로 learning rate을 나눠 주기 때문에

L2 regularization 텀이 포함 된 loss func를 전개해 보았을 때,

SGD에 적용한 것 보다 보다 decay rate이 더 작아질 가능성이 있는 것을 확인할 수 있다.

따라서 L2 regularization과 별도로 weight decay를 위한 텀을 수식에 추가하여 AdamW가 만들어졌는데 AdamW의 weight_decay 파라미터를 통해 decoupled_weight_decay를 적용할 수 있다.

In [118]:
from transformers import get_linear_schedule_with_warmup

In [119]:
model.train()
learning_rate = 5e-5
train_epoch = 4
optimizer = AdamW(model.parameters(), lr=learning_rate)
#*********
# t_total = len(train_loader)*train_epoch
# 스케줄러 알아볼 것
# scheduler = get_linear_schedule_with_warmup(optimizer, t_total/10, t_total) # 옵티마이저 설정 

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [120]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [121]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir = 'review')
os.makedirs('tc_model4', exist_ok = True)
step = 0

데이터로더에서 배치만큼 가져온 후 forward, backward pass를 수행한다.

In [122]:
# train_epoch = 1
lowest_valid_loss = 9999.
for epoch in range(train_epoch):
    with tqdm(train_loader, unit="batch") as tepoch:
        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)

            optimizer.zero_grad()

            output = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           token_type_ids=token_type_ids,
                           position_ids=position_ids,
                           labels=labels)

            loss = output.loss
            loss.backward()

            optimizer.step()
            # scheduler.step() # 스케줄러 설정

            # step += 1
            # writer.add_scalar('Train Loss', loss.item(),step)
            # del batch, start, end, start_logits, end_logits, loss

            tepoch.set_postfix(loss=loss.item())
            if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                # Evaluate the model five times per epoch
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                desc='Eval',
                                                                                                position=1,
                                                                                                leave=None):
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       position_ids=position_ids,
                                       labels=labels)

                        logits = output.logits
                        loss = output.loss
                        valid_losses.append(loss.item())

                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        predictions += batch_predictions
                        target_labels += batch_labels

                acc = compute_acc(predictions, target_labels)
                valid_loss = sum(valid_losses) / len(valid_losses)
                if lowest_valid_loss > valid_loss:
                    print('Acc for model which have lower valid loss: ', acc)
                    
                # writer.add_scalar('Eval Loss', loss, step)
                #wandb log 추적
                # metrics = {'accuracy' : acc, 'loss': valid_loss}
                # wandb.log(metrics)
    model.save_pretrained(f'tc_model4/model.{epoch}')


  0%|          | 0/3460 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch 0:  20%|██        | 694/3460 [01:54<28:14,  1.63batch/s, loss=0.154]

Acc for model which have lower valid loss:  0.9232308077019254


Epoch 0:  40%|████      | 1386/3460 [03:47<21:02,  1.64batch/s, loss=0.132]

Acc for model which have lower valid loss:  0.9354838709677419


Epoch 0:  60%|██████    | 2078/3460 [05:40<14:06,  1.63batch/s, loss=0.129]

...

Epoch 3:  60%|██████    | 2078/3460 [05:39<14:02,  1.64batch/s, loss=0.0461]

Acc for model which have lower valid loss:  0.940735183795949


Epoch 3:  80%|████████  | 2770/3460 [07:32<07:02,  1.63batch/s, loss=0.124]

Acc for model which have lower valid loss:  0.9374843710927732


Epoch 3: 100%|██████████| 3460/3460 [09:22<00:00,  6.15batch/s, loss=0.00496]


텐서보드 실행

tensorboard: error: invalid choice: 'logs/my_board/20220129-165343' (choose from 'serve', 'dev')의 오류가 있었으나 모델 수행 후 실행하니 작동됨

In [123]:
import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [124]:
log_dir = "/review/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [125]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [126]:
%tensorboard --logdir = {log_dir}

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-

# **모델 평가** #

In [127]:
model = BertForSequenceClassification.from_pretrained('tc_model4/model.3')
model.cuda()
model.eval() 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [128]:
import pandas as pd
test_df = pd.read_csv('sample_data/test_no_label.csv')

In [129]:
test_dataset = test_df['Id']

In [130]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    pre_data = [data_text_cleaning(line) for line in test_dataset]
    cnt = 0
    for x in pre_data:
      print(x)
    pre_data = [line for line in pre_data if line ]
    id_file_data = [tokenizer.encode(sent) for sent in pre_data]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [131]:
test = make_id_file_test(tokenizer, test_dataset)

['whole', 'new', 'experience', 'new', 'flavors', 'time', 'go']
['disappointing', 'old', 'favorite']
['authentic', 'thai', 'valley']
['sign', 'lease', 'people']
['nervous', 'made', 'feel', 'comfortable', 'welcome']
['thing', 'received', 'hot', 'coffee']
['ones', 'asu', 'ones', 'junior', 'high']
['bring', 'front', 'helpful']
['plus', 'dude', 'even', 'know', 'work', 'computer']
['excellent', 'selection', 'combos', 'choose']
['go', 'beyond']
['location', 'service', 'terrible']
['ordered', 'cesar', 'salad', 'side', 'blackened', 'chicken']
['nice', 'location', 'weekend', 'volleyball', 'tournaments']
['one', 'best', 'veal', 'dishes', 'ever', 'reasonable', 'prices']
['food', 'worse', 'find', 'freezer', 'section', 'walmart']
['tucson', 'native', 'place', 'brought', 'nostalgia', 'via', 'tastebuds']
['fun', 'watch', 'kitchen', 'rush', 'hour']
['came', 'sent', 'back']
['needless', 'say', 'back', 'mexican', 'food']
['stay', 'away', 'go', 'cheesecake', 'instead']
['highly', 'recommend', 'ahi', 'tuna

In [132]:
test[:10]

['101 2878 2047 3325 2047 26389 2051 2175 102',
 '101 15640 2214 5440 102',
 '101 14469 7273 3028 102',
 '101 3696 10084 2111 102',
 '101 6091 2081 2514 6625 6160 102',
 '101 2518 2363 2980 4157 102',
 '101 3924 100 3924 3502 2152 102',
 '101 3288 2392 14044 102',
 '101 4606 12043 2130 2113 2147 3274 102',
 '101 6581 4989 100 5454 102']

In [133]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [134]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [135]:
def collate_fn_sentiment_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)
    sorted_indices = np.array([len(input_id) for input_id in input_ids])

    input_ids = pad_sequence([torch.tensor(input_id) for input_id in input_ids],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_id) + [0] * (max_len - len(input_id)) for input_id in
         input_ids])
    token_type_ids = torch.tensor([[0] * len(input_id) for input_id in input_ids])
    position_ids = torch.tensor([list(range(len(input_id))) for input_id in input_ids])

    return input_ids, attention_mask, token_type_ids, position_ids

In [136]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_sentiment_test,
                                          num_workers=2)

In [137]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions


Test:  94%|█████████▍| 30/32 [00:00<00:00, 48.89it/s]
                                                     

In [138]:
test_df['Category'] = predictions

In [139]:
test_df.to_csv('submission_tc_model4.3.csv', index=False)